In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from google.colab import drive
import json
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import tarfile
import subprocess as sp



In [ ]:

import waac
import waac.config as config

In [ ]:
# drive.mount("/content/drive")

In [ ]:
import sys
# ROOT_DIR = Path("/content/drive/MyDrive/W&&C")
ROOT_DIR = config.ROOT_DIR
print(ROOT_DIR)
DATA_DIR = config.DATA_DIR
print(sys.path[-2:])
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [ ]:
DATA_DIR.mkdir(exist_ok=True, parents=True)
with tarfile.open(DATA_DIR / "nf_prize_dataset.tar.gz", "r:gz") as t:
  names = t.getnames()
print(names)

In [ ]:
text_files = [x for x in names if not (x.endswith(".tar") or x.endswith(".pl"))]
with tarfile.open(DATA_DIR / "nf_prize_dataset.tar.gz", "r:gz") as t:
  t.extractall(path=DATA_DIR, members = text_files)


In [ ]:
print((DATA_DIR / "download/README").read_text())

In [ ]:
non_text_files = [x for x in names if (x.endswith(".tar") or x.endswith(".pl"))]
with tarfile.open(DATA_DIR / "nf_prize_dataset.tar.gz", "r:gz") as t:
  t.extractall(path=DATA_DIR, members = non_text_files)

In [ ]:
metadata = """d2b86d3d9ba8b491d62a85c9cf6aea39        577547 movie_titles.txt
ed843ae92adbc70db64edbf825024514      10782692 probe.txt
88be8340ad7b3c31dfd7b6f87e7b9022      52452386 qualifying.txt
0e13d39f97b93e2534104afc3408c68c           567 rmse.pl
0098ee8997ffda361a59bc0dd1bdad8b    2081556480 training_set.tar"""
print(metadata)

metadata = metadata.split("\n")
metadata = [line.split() for line in metadata]
metadata = {z: {"md5sum": x, "size": int(y)} for x, y, z in metadata}

print(json.dumps(metadata, indent=2))

In [ ]:
# Compare the md5sum and size of the files in the download directory with the metadata from the README
for f in DATA_DIR.glob("download/*"):
  resp = sp.run(["md5sum", str(f)], capture_output=True)
  md5sum, file_name = resp.stdout.decode("utf8").split()
  file_name = Path(file_name).name
  if file_name in ["README"]:
    continue
  print(file_name)
  resp = sp.run(["ls", "-l", str(f)], capture_output=True)
  _, _, _, _, file_size, *_ = resp.stdout.decode("utf-8").split()
  file_size = int(file_size)
  assert metadata[file_name]["md5sum"] == md5sum
  assert metadata[file_name]["size"] == file_size
